# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

## Problem Overview
Let’s say your apartment lease is ending at the end of the year, and you need to find a new apartment. There are a lot of criteria you can use to help find a neighborhood you’d like to live in. One thing you care a lot about is a quiet neighborhood with a lot of greenery. 

Using NYC Open Data datasets and Zillow’s historic monthly rent averages, you will be creating a single Jupyter notebook to download, clean, and store data, as well as defining a set of SQL queries and visualizations to help answer questions of yours in search of a great area to live within your budget.


## Setup

In [23]:
# Import statements needed for the project
import math
from datetime import datetime, timedelta
from datetime import date
import numpy as np
import json
import pathlib
import urllib.parse
import psycopg2
import psycopg2.extras
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import seaborn as sns
import pandas as pd
import requests
import shapely
import sqlalchemy as db
import os
import folium

from shapely.geometry import Point
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, TIMESTAMP, Date, text

import re
import geoalchemy2 as gdb 
from geoalchemy2 import Geometry, WKTElement

import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import geoplot as gplt
import geoplot.crs as gcrs
from pathlib import PosixPath
from pyproj import CRS, Transformer
from shapely.ops import transform
from shapely.geometry import Point
from shapely.wkb import dumps
from pathlib import Path
import geodatasets
import contextily as cx

In [24]:
# Constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
COMPLAINTS_DATE_DIR =  pathlib.Path("data/311_data")
ZIPCODE_DATA_FILE = DATA_DIR /"nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

# Token Related Information Variable
APP_TOKEN = "WZUCCEUBzIBROFj20iUxnoyQV"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
NYC_DATA_311 = "erm2-nwe9.json"
NYC_DATA_TREES = "5rq2-4hqu.json"

DB_NAME = "Project4501"
DB_USER = "postgres"
DB_PASSWORD = "727403"
DB_HOST = 'localhost'
DB_URL=f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}'
DB_SCHEMA_FILE = "schema.sql"

crs='EPSG:4326'

rent_month_dict = {}
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [25]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

Four Dataset is downloaded and cleaned for part 1, they are NYC_311, NYC_ZILLOW, ZIPCODE and NYC_TREES
Here are some descriptions of the datasets:

1. `NYC_311` is the downloaded data where the 311 requests come from, we use the application token generated to download the data, since the whole data file is too large, we set a limit equals to 700000 and use `soql` command to store it in 30 JSON files. Through this we downloaded 1380551 data from the whole dataset as a population sample for this project.

2. `NYC_TREE` is the downloaded data from the tree census part, we adopt a similar method here, set a limit of 700000, but without the use of 30 JSON files, we got 683788 data here as a population sample for this project.

3. `NYC_ZILLOW` is the manually downloaded data from the shared Google file, which includes the historical monthly average rents by zip code from Zillow, we only keep the `zipcode`, `recorded_date` and `rent` variable as they are the only needed data for analysis in part 3 and part 4.

4. `NYC_ZIPCODE` is the manually downloaded data from the shared Google file, which includes the `shp` file for the geometry boundary of New York City.

### Download of NYC_311 request data

In [26]:
def download_nyc_311_data(url, force=False):
    json_files = [f"{i}.json" for i in range(30)]
    filenames = [COMPLAINTS_DATE_DIR / i for i in json_files ]
    
    if not COMPLAINTS_DATE_DIR.exists():
        COMPLAINTS_DATE_DIR.mkdir()
    
    limit = 700000
    for filename in filenames:
        if force or not filename.exists():
            print(f"Downloading")
            offset = int(filename.stem) * 700000
            all_entries = []
            selected_columns = ["unique_key", "created_date", "complaint_type", "incident_zip", "location"]
            soql_query311 = f"{url}?$$app_token={APP_TOKEN}&$select={','.join(selected_columns)}&$limit={700000}&$offset={offset}"
            response = requests.get(soql_query311)
            if response.status_code == 200: 
                entries = response.json()       
        
            with open(filename, "w") as f:
                json.dump(entries, f)
            print(f"Finished download")
            
    print(f"load data from {filenames} to {filenames}...")
    return filenames[:2]

### Download of NYC_Tree Data

In [27]:
def download_nyc_tree_data(url, force=False):
    url_path1 = urllib.parse.urlparse(url).path.split("/")[-1]
    filename = DATA_DIR / url_path1
    
    if not DATA_DIR.exists():
        DATA_DIR.mkdir()

    if force or not filename.exists():
        print(f"Downloing")
        
        limit = 700000
        all_entries = []
        offset = 0
        selected_columns = ["tree_id", "zipcode", "the_geom", "spc_common", "health", "status","latitude", "longitude"]
        end = 700000
        
        while offset < end:  
            total = limit+offset
            print(f"start from {offset} while total is {total}")
            soql_querytree = f"{url}?$$app_token={APP_TOKEN}&$select={','.join(selected_columns)}&$limit=700000&$offset={offset}"
            response = requests.get(soql_querytree)
            if response.status_code == 200: 
                entries = response.json()
                all_entries.extend(entries)
                offset += limit
                  
        with open(filename, "w") as f:
            json.dump(all_entries, f)
        print(f"Finished download")

    return filename

### Load and clean of the zipcode data

In [28]:
def load_and_clean_zipcodes(zipcode_datafile):
    global unique_zipcodes
    gdf = gpd.read_file(zipcode_datafile).to_crs(epsg=4326)
    unique_zipcodes_df = gdf[["ZIPCODE", "geometry"]].drop_duplicates(subset="ZIPCODE").reset_index(drop=True)
    unique_zipcodes = unique_zipcodes_df["ZIPCODE"]

    return unique_zipcodes_df

### Download and clean of the 311 data

In [29]:
def download_and_clean_311_data():
    filenames = download_nyc_311_data(f"{BASE_NYC_DATA_URL}{NYC_DATA_311}")
    processed_frames = []

    for filename in filenames:
        df = pd.read_json(filename, dtype="object")
        
        if "location" in df.columns:
            # Extract latitude and longitude
            df["latitude"] = df["location"].apply(lambda x: x.get("latitude") if isinstance(x, dict) else None)
            df["longitude"] = df["location"].apply(lambda x: x.get("longitude") if isinstance(x, dict) else None)
            
            # Create geometry for valid lat-long pairs
            df["geometry"] = [
                Point(float(lon), float(lat)) if pd.notna(lon) and pd.notna(lat) else None 
                for lat, lon in zip(df["latitude"], df["longitude"])
            ]

            # Drop unnecessary columns
            df.drop(columns=["location", "longitude", "latitude"], inplace=True)

            # Filter by incident zip and unique zipcodes
            df = df[pd.notna(df["incident_zip"]) & df["incident_zip"].isin(unique_zipcodes)]

            # Create a GeoDataFrame
            gdf = gpd.GeoDataFrame(df, geometry="geometry")
            gdf.crs = "EPSG:4326"
            
            processed_frames.append(gdf)

    # Concatenate all GeoDataFrames
    final_gdf = gpd.GeoDataFrame(pd.concat(processed_frames, ignore_index=True, sort=False)).drop_duplicates(subset=['unique_key']).reset_index(drop=True)
    return final_gdf


### Download and clean of the tree data

In [30]:
def download_and_clean_tree_data():
    filename = download_nyc_tree_data(f"{BASE_NYC_DATA_URL}{NYC_DATA_TREES}")
    tree_df = pd.read_json(filename, dtype='object')

    tree_df["geometry"] = tree_df["the_geom"].apply(lambda x: Point(x["coordinates"]) if x["coordinates"] else None)
    tree_df.drop(columns=["the_geom"], inplace=True)

    # Define the dtype conversion dictionary
    convert_dict = {
        "tree_id": int,
        "spc_common": str,
        "health": str,
        "status": str,
        "zipcode": str,
        "geometry": "geometry"  # Ensure geometry is identified for GeoDataFrame
    }

    # Convert dtypes and filter by zipcodes
    tree_gdf_normalized = gpd.GeoDataFrame(tree_df.astype(convert_dict))
    tree_gdf_normalized = tree_gdf_normalized[tree_gdf_normalized["zipcode"].isin(unique_zipcodes)]

    # Set the coordinate reference system
    tree_gdf_normalized.set_crs(epsg=4326, inplace=True)

    return tree_gdf_normalized


### Load and clean of the Zillow Data

In [31]:
def load_and_clean_zillow_data(zillow_datafile):
    data = pd.read_csv(zillow_datafile)
    
    zillow_data_NY = data[data.City == "New York"]
    numeric_columns = data.filter(regex="^\d").columns.tolist()
    zillow_NY_dataset = zillow_data_NY[["RegionName"] + numeric_columns]
    zillow_NY_dataset_halfclean = pd.melt(zillow_NY_dataset, id_vars=["RegionName"], var_name="recorded_date", value_name="rent")
    zillow_NY_dataset_clean = zillow_NY_dataset_halfclean.dropna(subset=["rent"])
    
    zillow_NY_dataset_clean = zillow_NY_dataset_clean.rename(columns={"RegionName": "zipcode"})
    zillow_NY_dataset_clean["recorded_date"] = pd.to_datetime(zillow_NY_dataset_clean["recorded_date"])
    
    return zillow_NY_dataset_clean

In [34]:
def load_all_data():
    df_zillow_data = load_and_clean_zillow_data(ZILLOW_DATA_FILE)
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data       
    )

In [35]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

load data from [PosixPath('data/311_data/0.json'), PosixPath('data/311_data/1.json'), PosixPath('data/311_data/2.json'), PosixPath('data/311_data/3.json'), PosixPath('data/311_data/4.json'), PosixPath('data/311_data/5.json'), PosixPath('data/311_data/6.json'), PosixPath('data/311_data/7.json'), PosixPath('data/311_data/8.json'), PosixPath('data/311_data/9.json'), PosixPath('data/311_data/10.json'), PosixPath('data/311_data/11.json'), PosixPath('data/311_data/12.json'), PosixPath('data/311_data/13.json'), PosixPath('data/311_data/14.json'), PosixPath('data/311_data/15.json'), PosixPath('data/311_data/16.json'), PosixPath('data/311_data/17.json'), PosixPath('data/311_data/18.json'), PosixPath('data/311_data/19.json'), PosixPath('data/311_data/20.json'), PosixPath('data/311_data/21.json'), PosixPath('data/311_data/22.json'), PosixPath('data/311_data/23.json'), PosixPath('data/311_data/24.json'), PosixPath('data/311_data/25.json'), PosixPath('data/311_data/26.json'), PosixPath('data/311_da

In [36]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ZIPCODE   248 non-null    object  
 1   geometry  248 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 4.0+ KB


In [37]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,ZIPCODE,geometry
0,11436,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."


In [38]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1380552 entries, 0 to 1380551
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype   
---  ------          --------------    -----   
 0   unique_key      1380552 non-null  object  
 1   created_date    1380552 non-null  object  
 2   complaint_type  1380552 non-null  object  
 3   incident_zip    1380552 non-null  object  
 4   geometry        1368209 non-null  geometry
dtypes: geometry(1), object(4)
memory usage: 52.7+ MB


In [39]:
geodf_311_data.head()

,unique_key,created_date,complaint_type,incident_zip,geometry
0,59596685,2023-12-01T12:00:00.000,Derelict Vehicles,10465,POINT (-73.82455 40.84384)
1,59602215,2023-12-01T01:03:37.000,Noise - Residential,10308,POINT (-74.15722 40.56203)
2,59599429,2023-12-01T01:03:29.000,Noise - Vehicle,11207,POINT (-73.90681 40.68472)
3,59598012,2023-12-01T01:03:27.000,Noise - Residential,10467,POINT (-73.86698 40.87631)
4,59598087,2023-12-01T01:03:20.000,Encampment,10024,POINT (-73.97900 40.78516)


In [40]:
geodf_tree_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 683788 entries, 0 to 683787
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   tree_id     683788 non-null  int64   
 1   zipcode     683788 non-null  object  
 2   spc_common  683788 non-null  object  
 3   health      683788 non-null  object  
 4   status      683788 non-null  object  
 5   latitude    683788 non-null  object  
 6   longitude   683788 non-null  object  
 7   geometry    683788 non-null  geometry
dtypes: geometry(1), int64(1), object(6)
memory usage: 47.0+ MB


In [41]:
geodf_tree_data.head()

,tree_id,zipcode,spc_common,health,status,latitude,longitude,geometry
0,180683,11375,red maple,Fair,Alive,40.72309177,-73.84421522,POINT (-73.84422 40.72309)
1,200540,11357,pin oak,Fair,Alive,40.79411067,-73.81867946,POINT (-73.81868 40.79411)
2,204026,11211,honeylocust,Good,Alive,40.71758074,-73.9366077,POINT (-73.93661 40.71758)
3,204337,11211,honeylocust,Good,Alive,40.71353749,-73.93445616,POINT (-73.93446 40.71354)
4,189565,11215,American linden,Good,Alive,40.66677776,-73.97597938,POINT (-73.97598 40.66678)


In [42]:
df_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9039 entries, 5 to 15224
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   zipcode        9039 non-null   int64         
 1   recorded_date  9039 non-null   datetime64[ns]
 2   rent           9039 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 282.5 KB


In [43]:
df_zillow_data.head()

,zipcode,recorded_date,rent
5,11226,2015-01-31,1944.609891
7,10025,2015-01-31,3068.951823
13,11206,2015-01-31,2482.829299
14,11221,2015-01-31,2125.738807
20,11235,2015-01-31,1687.789898


## Part 2: Storing Data

In [44]:
## first step: create database 'Project4501'in PostgreSQL
!createdb Project4501

createdb: error: database creation failed: ERROR:  database "Project4501" already exists


In [45]:
!psql --dbname Project4501 -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [46]:
def create_tables(engine):
    metadata = MetaData()

    nyc_311 = Table(
        'nyc_311', metadata,
        Column('unique_key', Integer, primary_key=True),
        Column('created_date', TIMESTAMP),
        Column('complaint_type', String),
        Column('incident_zip', Integer),
        Column('geometry', Geometry('POINT', srid=4326))
    )

    nyc_tree = Table(
        'nyc_tree', metadata,
        Column('tree_id', Integer, primary_key=True),
        Column('status', String),
        Column('health', String),
        Column('spc_common', String),
        Column('zipcode', Integer),
        Column('geometry', Geometry('POINT', srid=4326))
    )

    nyc_zipcode = Table(
        'nyc_zipcode', metadata,
        Column('zipcode', Integer, primary_key=True),
        Column('geometry', Geometry('POLYGON', srid=4326))
    )
    
    nyc_zillow = Table(
        'nyc_zillow', metadata,
        Column('zipcode', Integer, primary_key=True),
        Column('recorded_date', Date),
        Column('rent', Float)
    )

    metadata.create_all(engine)

In [47]:
def create_indexes(connection):
    sql_create_indexes = """
    CREATE INDEX IF NOT EXISTS idx_nyc_311_location ON "nyc_311" USING gist (geometry);
    CREATE INDEX IF NOT EXISTS idx_nyc_tree_location ON "nyc_tree" USING gist (geometry);
    CREATE INDEX IF NOT EXISTS idx_nyc_zipcode_location ON "nyc_zipcode" USING gist (geometry);
    """

    with connection.cursor() as cursor:
        cursor.execute(sql_create_indexes)

In [48]:
engine = create_engine(DB_URL)
create_tables(engine)

In [49]:
with psycopg2.connect(DB_URL) as connection:
    create_indexes(connection)

### Add Data to Database using SQLAlchemy


In [50]:
def write_dataframes_to_table(df_name, table_name):
    engine = create_engine(DB_URL)
    try:
        df_name.to_postgis(table_name, engine, if_exists='replace', index=False)
    except:
        df_name.to_sql(table_name, engine, if_exists='replace', index=False)

In [51]:
write_dataframes_to_table(geodf_311_data, 'nyc_311')
write_dataframes_to_table(geodf_tree_data, 'nyc_tree')
write_dataframes_to_table(geodf_zipcode_data,"nyc_zipcode")
write_dataframes_to_table(df_zillow_data,"nyc_zillow")